In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

# System call
os.system("")


# Class of different styles
class Style:
    BLACK = "\033[30m"  # Black
    RED = "\033[31m"  # Red (Used for error level in loguru)
    GREEN = "\033[32m"  # Green (Success messages)
    YELLOW = "\033[33m"  # Yellow (Warnings)
    BLUE = "\033[34m"  # Blue (Info)
    MAGENTA = "\033[35m"  # Magenta (Verbose or other custom levels)
    CYAN = "\033[36m"  # Cyan (Debug level in loguru)
    WHITE = "\033[37m"  # White (Default, standard text)

    UNDERLINE = "\033[4m"  # Underline for emphasis
    BOLD = "\033[1m"  # Bold text
    RESET = "\033[0m"  # Reset to default formatting

#### Connect to local llama_server

In [3]:
from ollama import Client
from IPython.display import display_markdown

client = Client(host="http://localhost:11434")

In [4]:
model_name = "qwen2.5-coder:7b"

In [5]:
# Pull a model
result = client.pull(model=model_name)
result

{'status': 'success'}

## Create reflection agent

In [6]:
generation_chat_history = []
reflection_chat_history = []

#### Pre-set the generation agent with a context and request

In [7]:
generation_prompt = {
    "role": "system",
    "content": "You are a experienced Python programmer tasked with generating high quality and very well documented Python code. If the user provides critique, respond with a revised version of your code from previous attempts.",
}

request_prompt = {
    "role": "user",
    "content": "Generate a python implementation of the quicksort algorithm.",
}

In [8]:
generation_chat_history.append(generation_prompt)
generation_chat_history.append(request_prompt)

#### Generate a response to the request given the context.

In [9]:
# Generate
generation_response = client.chat(model=model_name, messages=generation_chat_history)

generation_response_content = generation_response["message"]["content"]

# Assign the reponse the an entity called 'assistant'.
request_reponse_prompt = {
    "role": "assistant",
    "content": generation_response_content,
}

# Add the LLM response to the chat history.
generation_chat_history.append(request_reponse_prompt)

In [10]:
display_markdown(generation_response_content, raw=True)

Certainly! Below is a Python implementation of the Quicksort algorithm. This implementation includes detailed comments to help understand each part of the algorithm.

```python
def quicksort(arr):
    """
    Sorts an array in ascending order using the Quicksort algorithm.
    
    Parameters:
    arr (list): The list of elements to be sorted.
    
    Returns:
    list: The sorted list.
    """
    if len(arr) <= 1:
        return arr
    else:
        # Select a pivot element from the array. Here we choose the last element as the pivot.
        pivot = arr[-1]
        
        # Partitioning the array into three parts:
        # - less: elements less than the pivot
        # - equal: elements equal to the pivot
        # - greater: elements greater than the pivot
        less = [x for x in arr[:-1] if x < pivot]
        equal = [x for x in arr if x == pivot]
        greater = [x for x in arr[:-1] if x > pivot]
        
        # Recursively apply quicksort to the 'less' and 'greater' parts, then concatenate them with the 'equal' part.
        return quicksort(less) + equal + quicksort(greater)

# Example usage:
if __name__ == "__main__":
    example_array = [3, 6, 8, 10, 1, 2, 1]
    print("Unsorted array:", example_array)
    sorted_array = quicksort(example_array)
    print("Sorted array:", sorted_array)
```

### Explanation:

1. **Base Case**: If the array has 1 or 0 elements, it is already sorted, so we return it as is.
2. **Pivot Selection**: We choose the last element of the array as the pivot.
3. **Partitioning**:
   - We create three lists: `less` for elements less than the pivot, `equal` for elements equal to the pivot, and `greater` for elements greater than the pivot.
4. **Recursive Sorting**: We recursively apply the Quicksort algorithm to the `less` and `greater` lists.
5. **Concatenation**: Finally, we concatenate the sorted `less` list, the `equal` list, and the sorted `greater` list to form the final sorted array.

This implementation is simple and easy to understand but may not be the most efficient for large arrays due to the use of additional space for lists during partitioning. For an in-place version, consider using a different approach like Lomuto's or Hoare's partition scheme.

#### Pre-set the reflection agent with a context and request

In [11]:
reflection_prompt = {
    "role": "system",
    "content": "You are a experienced Python developer reponsible to supervise a software development project. You're tasked for creating and performing tests, identifying errors, and providing detailed feedback and recommendations to the user's code. Critique the code submitted for code quality, completeness, responsability, scope acucary and documentation.",
}

critique_prompt = {
    "role": "user",
    "content": generation_response_content,
}


reflection_chat_history.append(reflection_prompt)
reflection_chat_history.append(critique_prompt)

#### Generate a critique to the generated code.

In [12]:
critique_response = client.chat(model=model_name, messages=reflection_chat_history)
critique_response_content = critique_response["message"]["content"]

In [13]:
generation_chat_history.append({"role": "user", "content": critique_response_content})

In [14]:
generation_chat_history

[{'role': 'system',
  'content': 'You are a experienced Python programmer tasked with generating high quality and very well documented Python code. If the user provides critique, respond with a revised version of your code from previous attempts.'},
 {'role': 'user',
  'content': 'Generate a python implementation of the quicksort algorithm.'},
 {'role': 'assistant',
  'content': 'Certainly! Below is a Python implementation of the Quicksort algorithm. This implementation includes detailed comments to help understand each part of the algorithm.\n\n```python\ndef quicksort(arr):\n    """\n    Sorts an array in ascending order using the Quicksort algorithm.\n    \n    Parameters:\n    arr (list): The list of elements to be sorted.\n    \n    Returns:\n    list: The sorted list.\n    """\n    if len(arr) <= 1:\n        return arr\n    else:\n        # Select a pivot element from the array. Here we choose the last element as the pivot.\n        pivot = arr[-1]\n        \n        # Partitioni

In [15]:
best_response = client.chat(model=model_name, messages=generation_chat_history)

In [16]:
final_response = best_response["message"]["content"]

display_markdown(final_response, raw=True)

Thank you for your detailed critique! Based on your feedback, I have revised the Quicksort implementation to include an in-place version, input validation, and enhanced documentation.

Here is the improved version of the code:

```python
def quicksort(arr):
    """
    Sorts an array in ascending order using the Quicksort algorithm.
    
    Parameters:
    arr (list of int): The list of integers to be sorted.
    
    Returns:
    list of int: The sorted list of integers.
    
    Raises:
    TypeError: If the input is not a list.
    ValueError: If the list contains non-integer elements.
    
    Edge Cases:
    - An empty list returns an empty list.
    - A list with a single element returns the same list.
    """
    if not isinstance(arr, list):
        raise TypeError("Input must be a list")
    for element in arr:
        if not isinstance(element, int):
            raise ValueError("All elements in the list must be integers")
    
    def quicksort_in_place(low, high):
        if low < high:
            pi = partition(arr, low, high)
            quicksort_in_place(low, pi - 1)
            quicksort_in_place(pi + 1, high)

    def partition(low, high):
        pivot = arr[high]
        i = low - 1
        for j in range(low, high):
            if arr[j] < pivot:
                i += 1
                arr[i], arr[j] = arr[j], arr[i]
        arr[i + 1], arr[high] = arr[high], arr[i + 1]
        return i + 1

    quicksort_in_place(0, len(arr) - 1)
    return arr

# Example usage with in-place sorting:
if __name__ == "__main__":
    example_array = [3, 6, 8, 10, 1, 2, 1]
    print("Unsorted array:", example_array)
    sorted_array = quicksort(example_array)
    print("Sorted array:", sorted_array)
```

### Explanation of Improvements:

1. **In-place Quicksort**:
   - The `quicksort_in_place` function sorts the array in place, reducing the space complexity to O(log n).
   - The `partition` function rearranges elements around a pivot element.

2. **Input Validation**:
   - The function now checks if the input is a list and if all elements are integers, raising appropriate exceptions if not.

3. **Enhanced Documentation**:
   - The docstring has been enhanced to provide more detailed information about edge cases and potential usage scenarios.

4. **Example Usage**:
   - The example usage demonstrates how to sort an array using the in-place Quicksort implementation.

By incorporating these changes, the code is now more efficient, robust, and user-friendly.

In [17]:
reflection_chat_history

[{'role': 'system',
  'content': "You are a experienced Python developer reponsible to supervise a software development project. You're tasked for creating and performing tests, identifying errors, and providing detailed feedback and recommendations to the user's code. Critique the code submitted for code quality, completeness, responsability, scope acucary and documentation."},
 {'role': 'user',
  'content': 'Certainly! Below is a Python implementation of the Quicksort algorithm. This implementation includes detailed comments to help understand each part of the algorithm.\n\n```python\ndef quicksort(arr):\n    """\n    Sorts an array in ascending order using the Quicksort algorithm.\n    \n    Parameters:\n    arr (list): The list of elements to be sorted.\n    \n    Returns:\n    list: The sorted list.\n    """\n    if len(arr) <= 1:\n        return arr\n    else:\n        # Select a pivot element from the array. Here we choose the last element as the pivot.\n        pivot = arr[-1]\

### All in the loop

In [18]:
def user_query(question):
    generation_chat_history = [
        {
            "role": "system",
            "content": """
                Your task is to generate the most optimized and well-structured Python code based on the user's request.

                1. **Code Generation:** Create code that meets the user's requirements while ensuring it is efficient and follows best practices.

                2. **Feedback Incorporation:** If the user provides feedback or critique, revise your previous code accordingly. Ensure that the revised code reflects the user's input.

                3. **Quality Assurance:** Always maintain a focus on producing code of the highest quality, adhering to Python's PEP guidelines and prioritizing readability.
            """,
        },
        {
            "role": "user",
            "content": question,
        },
    ]
    reflection_chat_history = [
        {
            "role": "system",
            "content": """
                You are tasked with analyzing the user's provided Python code. You will give feedbacks and recommendations the the user's.

                1.Critique: If there are any issues, inefficiencies, or areas for improvement, provide a detailed list of specific critiques and actionable recommendations without including any Python code in your response.
                
                2.Focus Areas: Pay special attention to:
                    - Code readability
                    - Conformity to Python's PEP guidelines
                    - Quality of documentation (comments and docstrings)

                3.Conclusion: If the code is well-written and requires no changes, offer a few positive comments about its strengths, then output the token `<DONE>` exactly.
            """,
        }
    ]

    n = 10

    print(
        Style.YELLOW
        + Style.BOLD
        + f"\nUser:\n"
        + Style.RESET
        + Style.YELLOW
        + f"\t{question}\n"
    )

    for i in range(n):
        # Generate
        question_response = client.chat(
            model=model_name, messages=generation_chat_history
        )

        response_content = question_response["message"]["content"].replace("\n", "\n\t")

        print(
            Style.GREEN
            + Style.BOLD
            + f"\nAssistant:\n"
            + Style.RESET
            + Style.GREEN
            + f"\t{response_content}\n"
        )

        # Add the LLM response to the chat history.
        generation_chat_history.append(
            {
                "role": "assistant",
                "content": question_response["message"]["content"],
            }
        )

        reflection_chat_history.append(
            {
                "role": "user",
                "content": question_response["message"]["content"],
            }
        )

        # Critique
        critique_response = client.chat(
            model=model_name, messages=reflection_chat_history
        )

        critique_content = critique_response["message"]["content"].replace("\n", "\n\t")

        print(
            Style.YELLOW
            + Style.BOLD
            + f"\nUser:\n"
            + Style.RESET
            + Style.YELLOW
            + f"\t{critique_content}\n"
        )

        if "<DONE>" in critique_response["message"]["content"]:
            break

        generation_chat_history.append(
            {
                "role": "user",
                "content": critique_response["message"]["content"],
            }
        )

        reflection_chat_history.append(
            {
                "role": "assistant",
                "content": critique_response["message"]["content"],
            }
        )

    print(
        Style.BLUE
        + Style.BOLD
        + f"\nUser:\n"
        + Style.RESET
        + Style.BLUE
        + f"\t{response_content}\n"
    )

    return generation_chat_history, reflection_chat_history

In [19]:
a, b = user_query("Generate a python implementation of the quicksort algorithm")


User:
	Generate a python implementation of the quicksort algorithm


Assistant:
	Certainly! Below is an optimized and well-structured Python implementation of the Quicksort algorithm:
	
	```python
	def quicksort(arr):
	    """
	    Sorts an array using the Quicksort algorithm.
	
	    Args:
	        arr (list): The list of elements to be sorted.
	
	    Returns:
	        list: The sorted list.
	    """
	    if len(arr) <= 1:
	        return arr
	    else:
	        pivot = arr[len(arr) // 2]
	        left = [x for x in arr if x < pivot]
	        middle = [x for x in arr if x == pivot]
	        right = [x for x in arr if x > pivot]
	        return quicksort(left) + middle + quicksort(right)
	
	# Example usage:
	if __name__ == "__main__":
	    array = [3, 6, 8, 10, 1, 2, 1]
	    sorted_array = quicksort(array)
	    print("Sorted Array:", sorted_array)
	```
	
	### Explanation:
	
	1. **Base Case**: If the list has 1 or no elements, it is already sorted, so we return it as is.
	2. **Pivot Sel